In [1]:
import pandas as pd
import numpy as np
import os

---

In [2]:
fullContests = pd.read_csv('data/WorkingData.csv')
durations = pd.read_csv('data/Durations.csv').drop(columns=['Unnamed: 0'])

In [50]:
def combineChunkPreds(chunkName):
    dirList = os.listdir("data/results/"+chunkName+"/")
    chunkResults = []
    for filename in dirList[:]: #limit to .csv
        if(filename[-4:] == '.csv'):
            version = filename.split('_')[2]
            load = pd.read_csv('data/results/'+chunkName+'/'+filename).drop(columns=['Unnamed: 0'])
            load.columns = ['ContestId', version, "label"]
            load = load.drop(columns=['label'])
            chunkResults.append(load)
    combinedDF = chunkResults[0]
    for nextDF in chunkResults[1:]:
        combinedDF = pd.merge(combinedDF, nextDF, on='ContestId', how='left')
    return combinedDF


In [63]:
dirList = os.listdir("data/results/")
kalmanResults = []
for filename in dirList: #limit to .csv
    if(filename[0] != '.' and filename[0] != '_' ):
#         print(filename)
        r = combineChunkPreds(filename)
        kalmanResults.append(r)
#         kalmanResults.append(pd.read_csv('data/results/'+filename).drop(columns=['Unnamed: 0']))
resultsDF = pd.concat(kalmanResults).set_index('ContestId').dropna(axis=1)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  if __name__ == '__main__':


In [65]:
len(resultsDF)

142565

In [5]:
# cids = resultsDF['ContestId'].unique()
# combinedObjs = []
# count = 0
# for cid in cids[:]:
#     obj = {'ContestId':cid}
#     count+=1
#     duration = durations[durations['ContestId']==cid].iloc[0].Duration
#     obj['Duration']=duration
#     for v in versions:
#         currC = resultsDF[resultsDF['ContestId']==cid]
#         AB = str(currC[currC['label']==v].Xf.iloc[0])[1:-1].split(', ')
        
#         obj['A'+v]=AB[0]
#         obj['B'+v]=AB[1]
#     if(count%10000==0):
#         print(str(count)+".../..."+str(len(cids)))
# #         print(obj)
#     combinedObjs.append(obj)

In [6]:
# combinedDF = pd.DataFrame(data = combinedObjs).set_index('ContestId')

In [7]:
# combinedDF.to_csv('data/QR_Results.csv')

---

In [8]:
# qrDF = pd.read_csv('data/QR_Results.csv').set_index('ContestId')
# qrDFCols = qrDF.columns
# dropList = []
# vNames = []
# for col in qrDFCols:
#     if("Pred" in col):
#         dropList.append(col)
#     if(col[0]=="A"):
#         vNames.append(col[1:])
# qrDF = qrDF.drop(columns = dropList)

In [9]:
# successDF = pd.read_csv('data/WorkedData.csv').set_index('ContestId')[['Success']]
# contestsDF = pd.read_csv('data/WorkingData.csv').set_index('ContestId')[['EntryFeeAmount', 'TotalPrizeAmount', 'MaxNumberPlayers']]
# contestsDF = pd.merge(contestsDF, successDF, on='ContestId', how='left')

In [10]:
# resultsDF = pd.merge(qrDF, contestsDF, on='ContestId', how='left').dropna()